In [ ]:

def ask_question(question, answers):
    while True:
        answer = input("{} + ({})".format(question, answers))
        if answer not in answers:
            print("Invalid answer")
        else:
            return answer
        
mode = ask_question("What gamemode are you playing?", ["ap", "cm"])

print(mode)

In [2]:
import argparse

import mcts
import mcts_transpositions
import util
from gamestate import State


# model = util.load_model('logisticregression')
# print model
# print util.all_models

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
119
120


In [3]:


def ask_question(question, answers):
    while True:
        answer = input("{} + ({})".format(question, answers))
        if answer not in answers:
            print("Invalid answer")
        else:
            return answer

def get_pick(state, pick_ban, count):
    while True:
        skip = False
        hero_names = input('{} {} hero: '.format('Choose', count)).split(',')
        heroes = list()
        for name in hero_names:
            id = util.simple_heroes.approximate_name_to_ordered(name)
            if id is None:
                print('Could not find hero {}'.format(name))
                skip = True
                break
            elif id in state.radiant_heroes or id in state.dire_heroes or id in state.banned_heroes:
                print("Hero {} already picked or banned.".format(name))
                skip = True
                break
            else:
                heroes.append(id)
        if skip: continue
        if len(heroes) != count:
            print('Wrong number of heroes')
            continue
        return tuple(heroes)        

def print_state(state):
        print('Banned:', [util.simple_heroes.ordered_to_name(i) for i in state.banned_heroes])
        print('Radiant:', [util.simple_heroes.ordered_to_name(i) for i in state.radiant_heroes])
        print('Dire:', [util.simple_heroes.ordered_to_name(i) for i in state.dire_heroes])
    
    

def real_game(modelname, time_limit, recommendation_count):
    mode = ask_question("What gamemode are you playing?", ["ap", "cm"])
    side = ask_question("Which side are you playing on?", ["radiant", "dire"])
    first = ask_question("Do you have first pick / ban?", ["y", "n"])
    
    if mode == "ap":
        util.pick_ban_order = util.allpick_order
    else:
        util.pick_ban_order = util.cm_order
        
    radiant_goes_first = (side == "radiant" and first == "y") or (side == "dire" and first == "n")
    node = mcts_transpositions.Node(mcts.State(radiant_goes_first))
    transpositions = dict()
    model = util.load_model(modelname)
    players_turn = (side == "radiant" and node.state.radiant_moves_next) or (side == "dire" and not node.state.radiant_moves_next)
    while not node.state.is_terminal():
        print_state(node.state)
        choices = node.state.get_actions()
        choices_sets = [set(i) for i in choices]
        
        (pick_ban, count) = util.pick_ban_order[node.state.pick_ban_position]
        subject = 'pick' if pick_ban == util.pick else 'ban'
        print("The next action is a", subject, "of", count, "heroes.")
        
        if players_turn:
            print("It is your turn. MCTS recommends the following heroes: ...")
            (_, root_node, transpositions) = mcts_transpositions.uct_search(model,initial_node=node, time_limit=time_limit, transpositions=transpositions)
            node = root_node
            def to_transpo(n): return transpositions[mcts_transpositions.state_to_key(n.state)]
            children = sorted(root_node.children, key=lambda n: to_transpo(n).total_simulated_reward / to_transpo(n).visit_count, reverse=True)
            for c in children[:recommendation_count]:
                print([util.simple_heroes.ordered_to_name(i) for i in c.incoming_action], to_transpo(c).total_simulated_reward / to_transpo(c).visit_count, to_transpo(c).visit_count)
        else:
            print("It is the other team's turn. What did they do?")
        players_turn = not players_turn
        choice = get_pick(node.state, pick_ban, count)
        
        print()
        assert(set(choice) in choices_sets)
        found = False
        for n in node.children:
            if n.incoming_action == choice:
                node = n
                node.parent = None
                node.incoming_action = None
                found = True
        if not found:
            node = mcts.Node(node.state.get_next_state(choice))
    print('Done!')
    print_state(node.state)
    print('Predicting Radiant win probability with all models:')
    for model_name in util.all_models:
        model = util.load_model(model_name)
        print(model_name,':', util.predict_radiant_win_probability(util.state_to_feature(node.state), model))


models = ask_question("Please enter the model name ",["gradient_Boosting_classifier","K_Neighbhors_classifier","logisticregression","random_forest_classifier"])
print(models)
recommendation_count = 10
time_limit = 1
real_game(models,time_limit,recommendation_count)

Please enter the model name  + (['gradient_Boosting_classifier', 'K_Neighbhors_classifier', 'logisticregression', 'random_forest_classifier'])'logisticregression'
logisticregression
What gamemode are you playing? + (['ap', 'cm'])'ap'
Which side are you playing on? + (['radiant', 'dire'])'radinat'
Invalid answer
Which side are you playing on? + (['radiant', 'dire'])'radiant'
Do you have first pick / ban? + (['y', 'n'])'y'
('Banned:', [])
('Radiant:', [])
('Dire:', [])
('The next action is a', 'pick', 'of', 1, 'heroes.')
It is your turn. MCTS recommends the following heroes: ...
(['ursa'], 0.53013197343918506, 313)
(['enchantress'], 0.52720891486955745, 218)
(['abaddon'], 0.52680554837481464, 185)
(['vengefulspirit'], 0.52605607860730552, 151)
(['lycan'], 0.51216259671096387, 38)
(['jakiro'], 0.51142182690981008, 20)
(['visage'], 0.50939833603733231, 25)
(['weaver'], 0.50632978415368646, 12)
(['meepo'], 0.50452592290120601, 10)
(['oracle'], 0.50255012280663802, 8)
Choose 1 hero: 'enchant

In [ ]:
for model_name in util.all_models:
    print model_name